In [ ]:
import pandas as pd
import numpy as np
from readFile import split_into_values, toRPdata

In [ ]:
# columns 와 value는 사용자 입력
#df = pd.read_csv('resources/AXISX_resample.csv')
df = pd.read_csv('resources/CLAMP_resample.csv')
value = ['value']
#df = pd.read_csv('resources/Dataset1.csv')
columns = ['chip', 'wire', 'segment']
value = ['value']

In [ ]:
df = df.loc[:, columns + value] #('chip', 'wire', 'value')는 사용자 입력
size = 96 # 모델에 따라 변경 28, 96
result = split_into_values(df, columns)

In [ ]:
# dataframe -> list
result_list = result.values.tolist()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

def MinMax(data):
    MMS = MinMaxScaler().fit(data)
    scaled = MMS.transform(data)
    return scaled

In [ ]:
# result_list transpose
result_T = [list(x) for x in zip(*result_list)]

In [ ]:
# minmax 정규화
result_scaled = MinMax(result_T)

In [ ]:
# 다시 result transpose 해서 원래대로
result_scaled = [list(x) for x in zip(*result_scaled)]

In [ ]:
from tslearn.preprocessing import TimeSeriesScalerMeanVariance, TimeSeriesResampler

In [ ]:
# 2. 시계열 셋 크기 변경
result_ = TimeSeriesResampler(sz=size).fit_transform(result)

In [ ]:
data = result_.reshape(result_.shape[0], 1, size)
X = toRPdata(data, threshold='point', percentage=30) # rp 1
# X = toRPdata(data) # rp 2

In [ ]:
X_scaled = np.expand_dims(X, axis=3)
X_scaled.shape

In [ ]:
from keras.models import Sequential

In [ ]:
from keras.models import load_model

In [ ]:
model = load_model('CLAMP_96_Autoencoder_128_3e-05_weight.h5')

In [ ]:
print(model.summary())

In [ ]:
from keras import backend as K

compressed_layer = 8
get_layer_output = K.function([model.layers[0].input],[model.layers[compressed_layer].output])
compressed = get_layer_output([X_scaled])[0]
feature = compressed.reshape(compressed.shape[0], compressed.shape[1]*compressed.shape[2]*compressed.shape[3])

print(feature)
print(feature.shape)

In [ ]:
print(compressed.shape)

In [7]:
# Clustering Algorithm
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering, AgglomerativeClustering

# Normalization
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

# Needed Library!
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [8]:
# Accuracy를 가져오는 함수
# data_y: 정답 데이터
# pred_y: 예측 데이터
def getAccuracy(data_y, pred_y):
    count = 0
    bool_array = (data_y == pred_y)
    for correct in bool_array:
        if(correct):
            count += 1
    return count / pred_y.size

In [9]:
# 정답 레이블 만들기
## 원하는 클래스 레이블의 리스트를 넘긴다.
## ex) list = [1, 4, 5] -> 클래스 1번, 4번, 5번 에 대하여 레이블 생성
def getClassLabelFor(list, batch_size=190):
    y=np.array([])
    for i in list:
        y_=np.full((1, batch_size), i)[0]
        y=np.hstack([y, y_])

    return y

In [10]:
# Cluster Algorithm
def kmeans(dataset, n_clusters, normalization='standard'):

    scaled_dataset = []
    if normalization == 'standard':
        scaler = StandardScaler().fit(dataset)
        scaled_dataset = scaler.transform(dataset)
    elif normalization == 'minmax':
        scaler = MinMaxScaler().fit(dataset)
        scaled_dataset = scaler.transform(dataset)
    else:
        scaled_dataset = dataset
        print("정규화 진행 안함")
    print("Scaled_dataset: \n{}".format(scaled_dataset))
    cluster_data = KMeans(n_clusters=n_clusters).fit(scaled_dataset)
    return cluster_data, scaled_dataset

# DBSCAN
def dbscan(dataset, eps=0.5, min_samples=5, normalization='standard'):
   
    scaled_dataset = []
    if normalization == 'standard':
        scaler = StandardScaler().fit(dataset)
        scaled_dataset = scaler.transform(dataset)
    elif normalization == 'minmax':
        scaler = MinMaxScaler().fit(dataset)
        scaled_dataset = scaler.transform(dataset)
    else:
        scaled_dataset = dataset
        print("정규화 진행 안함")

    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    cluster_data = dbscan.fit_predict(scaled_dataset)
    return cluster_data, scaled_dataset

# Spectral Clustering
def spectralClustering(dataset, n_clusters, n_init = 10, normalization='standard'):
    
    #scaled_dataset = norm(dataset, normalization=normalization)
    scaled_dataset = []
    if normalization == 'standard':
        scaler = StandardScaler().fit(dataset)
        scaled_dataset = scaler.transform(dataset)
    elif normalization == 'minmax':
        scaler = MinMaxScaler().fit(dataset)
        scaled_dataset = scaler.transform(dataset)
    else:
        scaled_dataset = dataset
        print("정규화 진행 안함")
    cluster_data = SpectralClustering(n_clusters=n_clusters, n_init=n_init).fit_predict(scaled_dataset)
    return cluster_data, scaled_dataset

# Hierarchical Clustering
def hierarchicalClustering(dataset, n_clusters, n_init = 10, linkage = 'ward', normalization='standard'):

    #scaled_dataset = norm(dataset, normalization=normalization)
    scaled_dataset = []
    if normalization == 'standard':
        scaler = StandardScaler().fit(dataset)
        scaled_dataset = scaler.transform(dataset)
    elif normalization == 'minmax':
        scaler = MinMaxScaler().fit(dataset)
        scaled_dataset = scaler.transform(dataset)
    else:
        scaled_dataset = dataset
        print("정규화 진행 안함")
    cluster_data = AgglomerativeClustering(n_clusters = n_clusters, linkage = linkage )
    return cluster_data, scaled_dataset

In [ ]:
import Clustering as C
import Cluster as c

In [ ]:
result, scaled_x = c.kmeans(feature, 6, normalization='minmax')

In [ ]:
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)
print(result.labels_)

In [ ]:
y_label = getClassLabelFor([0, 1, 2, 3, 4, 5])
print(y_label)

In [ ]:
i = 0
for c in result.labels_:
    if c == 2:
        result.labels_[i] = 0
    elif c == 1:
        result.labels_[i] = 1
    elif c == 5:
        result.labels_[i] = 2
    elif c == 4:
        result.labels_[i] = 3
    elif c == 0:
        result.labels_[i] = 4
    elif c == 3:
        result.labels_[i] = 5
    i += 1

In [ ]:
class_ = np.array(result.labels_)
accuracy = getAccuracy(y_label, class_)

In [ ]:
print(accuracy)

In [11]:
result2, scaled_x2 = dbscan(feature)

In [ ]:
print(scaled_x2)

In [12]:
import sys
import numpy

numpy.set_printoptions(threshold=sys.maxsize)
print("Clusters DBSCAN: \n{}".format(scaled_x2))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
numpy.save('./compressed', compressed) # x_save.npy

In [ ]:
numpy.save('./feature', feature) # x_save.npy

In [ ]:
numpy.save('./X_scaled', X_scaled) # x_save.npy

In [ ]:
df = numpy.hstack([x, dbscan_x.reshape(-1, 1)]) # x_scaled_ss 오른쪽

In [1]:
import numpy as np

In [2]:
compressed = np.load('./compressed.npy')

In [3]:
print(compressed)

[[[[2.1255448  0.93027747]
   [2.7712724  0.24135658]
   [3.832439   0.77795386]
   ...
   [3.7606878  0.5797688 ]
   [3.7606878  0.5797688 ]
   [3.761414   0.56892085]]

  [[2.31936    0.27294186]
   [2.3148901  2.2734265 ]
   [4.295268   0.74573934]
   ...
   [4.0125594  0.3670555 ]
   [4.0125594  0.3670555 ]
   [4.0131826  0.35865983]]

  [[4.113431   0.61970854]
   [3.5518453  1.0416329 ]
   [1.5736517  1.3612006 ]
   ...
   [4.0125594  0.3670555 ]
   [4.0125594  0.3670555 ]
   [4.0131826  0.35865983]]

  ...

  [[4.014099   0.5525328 ]
   [4.0125594  0.3670555 ]
   [4.0125594  0.3670555 ]
   ...
   [0.47503543 0.78421843]
   [1.1137333  1.0478398 ]
   [1.4541966  1.2668641 ]]

  [[4.014099   0.5525328 ]
   [4.0125594  0.3670555 ]
   [4.0125594  0.3670555 ]
   ...
   [1.1212106  1.0299368 ]
   [0.68415475 0.9009719 ]
   [0.5457814  0.77407324]]

  [[3.9232059  0.4530573 ]
   [3.9197192  0.25944814]
   [3.9197192  0.25944814]
   ...
   [1.1210308  0.9377413 ]
   [0.9910853  0.494881

In [4]:
feature = np.load('./feature.npy')

In [5]:
X_scaled = np.load('./X_scaled.npy')